**Aisha Muhammad Nawaz  L200921**  
PySpark QUIZ # 2
8A BSCS MMD  
19th March 2024

Instructions:
Sentimental Analysis on Tweets
3 files given: tweets,negative,positive
Output count and categorized tweets in two files, ignore words whose length is less than 4


In [2]:
# #Running on Colab
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=4f3387353f2c2e7e095aa5204de3fb8425dc23ea4ed1a7269d4659adb01995ff
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum fonts-ipafont-gothic
  fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 39.7 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package

In [ ]:
!sudo apt update

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [810 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,695 kB]
Get:13 http://security.ubuntu.com/ubunt

In [3]:
#  Import the libraries we will need
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

# Create Spark session and ContextRun PySpark.
# create the session
conf = SparkConf().set("spark.ui.port","4050")
# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.appName("DataFrame").config('spark.ui.port', '4050').getOrCreate()
spark

In [ ]:
tweets = sc.parallelize(['Wow! The weather is great today','This instructor hates students','I was not this bad at maths before','She sucks at lying'])
negative=list(sc.parallelize(['bad','hates','sucks']).collect())
positive=list(sc.parallelize(['great']).collect())

def findCat(line,pos,neg):
  pC=0
  pN=0
  for word in line:
    if(len(word)>4):
      if(word in pos):
        pC=pC+1
      elif(word in neg):
        pN=pN+1
  if(pC>pN):
    return 'Positive'
  else:
    return 'Negative'
tweetsCat=tweets.map(lambda x: (x,findCat(x.split(' '),negative,positive)))
tweetsCount=tweetsCat.map(lambda x: (x[1],x[0])).countByKey()

tweetsCat.saveAsTextFile('Category3.txt')
sc.parallelize([(s,c) for s,c in tweetsCount.items()]).saveAsTextFile('Count3.txt')

In [ ]:
tweets = sc.parallelize(['Wow! The weather is great today', 'This instructor hates students', 'I was not this bad at maths before', 'She sucks at lying'])
negative = sc.parallelize(['bad', 'hates', 'sucks']).collect()
positive = sc.parallelize(['great']).collect()

def findCat(line, negative, positive):
    nC = 0
    nP = 0
    for w in line:
        w = w.lower().strip('.,?!')  # Convert to lowercase and remove punctuation
        if len(w) > 4:
            if w in negative:
                nC += 1
            elif w in positive:
                nP += 1
    if nP > nC:
        return 'Positive'
    else:
        return 'Negative'

tweetsCat = tweets.map(lambda x: (x, findCat(x.lower().split(), negative, positive)))
tweetsCount = tweetsCat.map(lambda x: (x[1], 1)).reduceByKey(lambda x,y:x+y)

tweetsCat.saveAsTextFile('Category5.txt')
tweetsCount.saveAsTextFile('Count5.txt')



Try Before Final Exam

In [5]:
tweets=sc.textFile('TWEETS.txt')
PositiveWords=sc.broadcast(set(sc.textFile('Positive_words.txt').collect()))
NegativeWords=sc.broadcast(set(sc.textFile('Negative_words.txt').collect()))

def classifyTweet(line,PositiveWords,NegativeWords):
  words=[w for w in line.split(" ") if len(w)>=4]
  positiveCount=0
  negativeCount=0
  for word in words:
    word=word.lower().replace('!','').replace('.','').replace(',','')
    if (word) in list(PositiveWords):
      positiveCount=positiveCount+1
    elif(word in list(NegativeWords)):
      negativeCount=negativeCount+1


  if(positiveCount>=negativeCount):
    return (line,'Positive')

  else:
    return (line,'Negative')


classifiedTweets=tweets.map(lambda line: classifyTweet(line,PositiveWords.value,NegativeWords.value))
counts=classifiedTweets.map(lambda line: (line[1]+' tweets count: ',1)).reduceByKey(lambda x,y: x+y)

classifiedTweets.saveAsTextFile('TweetSentimentAnalysis4.txt')
counts.saveAsTextFile('TweetsCount2.txt')